In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json
from pandas import json_normalize
import re
from datetime import datetime

load_dotenv

token = os.getenv("API_TOKEN")

headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application.json'
}
response = requests.get('https://api.clashofclans.com/v1/clans/%232GLLPQYLL/currentwar', headers=headers)
response = response.json()

In [2]:
dt = datetime.now()
# if response['state'] == 'inWar':
#     with open(f'war{dt.month}.{dt.day}.{dt.year}time{dt.hour}.{dt.minute}.json', "a") as json_file:
#         json.dump(response, json_file, indent=4)
# elif response['state'] == 'warEnded':
#     with open(f'war{dt.month}.{dt.day}.{dt.year}ended.json', "a") as json_file:
#         json.dump(response, json_file, indent=4)


progress_directory = '/home/lukethan/projects/clash_project/in_progress'
finished_directory = '/home/lukethan/projects/clash_project/finished'

# Create the absolute file path
file_path_p = os.path.join(progress_directory, f'war{dt.month}.{dt.day}.{dt.year}time{dt.hour}.{dt.minute}.json')

if response["state"] == 'inWar':
    with open(file_path_p, "a") as json_file:
        json.dump(response, json_file, indent=4)
        logging.info(f'Response recorded for inWar at {dt}')
elif response["state"] == 'warEnded':
    end_time = response['endTime'][0:8]
    file_name = f'war{end_time}.json'
    file_path = os.path.join(finished_directory, file_name)
    if os.path.exists(file_path):
        logging.info(f'{file_path} already exists')
    else:
        with open(file_path, "a") as json_file:
            json.dump(response, json_file, indent=4)
        logging.info(f"{file_path} created and response recorded at {dt}")

NameError: name 'dt' is not defined

In [4]:
data = json_normalize(response, record_path=['clan', 'members'])
data

,tag,name,townhallLevel,mapPosition,opponentAttacks,bestOpponentAttack.attackerTag,bestOpponentAttack.defenderTag,bestOpponentAttack.stars,bestOpponentAttack.destructionPercentage,bestOpponentAttack.order,bestOpponentAttack.duration,attacks
0,#UGRL8LV8,mcnasty,13,3,1,#PCUGCRG,#UGRL8LV8,3,100,5,121,NaN
1,#8Q8UQL8Y,NMbaseball22,12,6,1,#PCUGCRG,#8Q8UQL8Y,3,100,6,69,NaN
2,#8CRG2JR9,MeekMulli,12,4,1,#PG8UYYYG2,#8CRG2JR9,3,100,1,96,"[{'attackerTag': '#8CRG2JR9', 'defenderTag': '..."
3,#G222UU009,Ben Dover,12,7,1,#G8RLLL2YR,#G222UU009,3,100,13,84,NaN
4,#9PRCGP9Y,Luke,15,2,2,#YYRP8U8VY,#9PRCGP9Y,3,100,8,87,NaN
5,#9GGVU8C8,Shamrock 2.0,16,1,2,#YYRP8U8VY,#9GGVU8C8,2,75,7,63,"[{'attackerTag': '#9GGVU8C8', 'defenderTag': '..."
6,#GYQ09LYRR,winston,12,5,1,#PG8UYYYG2,#GYQ09LYRR,3,100,2,92,"[{'attackerTag': '#GYQ09LYRR', 'defenderTag': ..."
7,#LPG2LYYGJ,Drippy,11,8,1,#URUQ92R,#LPG2LYYGJ,3,100,3,95,"[{'attackerTag': '#LPG2LYYGJ', 'defenderTag': ..."
8,#GGV0JPQ2C,MewThree,10,10,1,#G8RLLL2YR,#GGV0JPQ2C,3,100,14,60,NaN
9,#GPRURPLRG,Wreckit Ralph,11,9,1,#URUQ92R,#GPRURPLRG,3,100,4,107,"[{'attackerTag': '#GPRURPLRG', 'defenderTag': ..."


In [5]:
members_df = json_normalize(response, record_path=['clan', 'members'], meta=[])
total_members_df = members_df.copy()


In [6]:
attacks_df = json_normalize(
    members_df.to_dict(orient='records'),  # Convert DataFrame to dicts
    record_path='attacks',
    meta=['name'],
    meta_prefix='member_'
)
attacks_df

,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name
0,#8CRG2JR9,#QR9GLUCLP,1,91,18,161,MeekMulli
1,#9GGVU8C8,#2QQ9RQJ0P,2,97,16,176,Shamrock 2.0
2,#GYQ09LYRR,#URUQ92R,2,94,11,129,winston
3,#GYQ09LYRR,#QR9GLUCLP,1,75,17,157,winston
4,#LPG2LYYGJ,#G28LQQGL8,3,100,10,93,Drippy
5,#GPRURPLRG,#99PY8V9C2,3,100,9,142,Wreckit Ralph


In [7]:
attacks_df['attack_num'] = attacks_df.groupby('member_name').cumcount() + 1
attacks_df



,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num
0,#8CRG2JR9,#QR9GLUCLP,1,91,18,161,MeekMulli,1
1,#9GGVU8C8,#2QQ9RQJ0P,2,97,16,176,Shamrock 2.0,1
2,#GYQ09LYRR,#URUQ92R,2,94,11,129,winston,1
3,#GYQ09LYRR,#QR9GLUCLP,1,75,17,157,winston,2
4,#LPG2LYYGJ,#G28LQQGL8,3,100,10,93,Drippy,1
5,#GPRURPLRG,#99PY8V9C2,3,100,9,142,Wreckit Ralph,1


In [8]:
attacks_df['total_attacks'] = attacks_df.groupby('member_name')['attack_num'].transform('max')
attacks_df

,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num,total_attacks
0,#8CRG2JR9,#QR9GLUCLP,1,91,18,161,MeekMulli,1,1
1,#9GGVU8C8,#2QQ9RQJ0P,2,97,16,176,Shamrock 2.0,1,1
2,#GYQ09LYRR,#URUQ92R,2,94,11,129,winston,1,2
3,#GYQ09LYRR,#QR9GLUCLP,1,75,17,157,winston,2,2
4,#LPG2LYYGJ,#G28LQQGL8,3,100,10,93,Drippy,1,1
5,#GPRURPLRG,#99PY8V9C2,3,100,9,142,Wreckit Ralph,1,1


In [9]:
total_attacks_df = attacks_df[['member_name', 'total_attacks']].drop_duplicates().set_index('member_name')
total_attacks_df

,total_attacks
member_name,
MeekMulli,1
Shamrock 2.0,1
winston,2
Drippy,1
Wreckit Ralph,1


In [10]:
pivot_df = attacks_df.pivot_table(
    index=['member_name'], 
    columns='attack_num', 
    values=['attackerTag', 'defenderTag', 'stars', 'destructionPercentage', 'order', 'duration'], 
    aggfunc='first'
)
pivot_df

attackerTag             defenderTag              \
attack_num              1           2           1           2   
member_name                                                     
Drippy         #LPG2LYYGJ         NaN  #G28LQQGL8         NaN   
MeekMulli       #8CRG2JR9         NaN  #QR9GLUCLP         NaN   
Shamrock 2.0    #9GGVU8C8         NaN  #2QQ9RQJ0P         NaN   
Wreckit Ralph  #GPRURPLRG         NaN  #99PY8V9C2         NaN   
winston        #GYQ09LYRR  #GYQ09LYRR    #URUQ92R  #QR9GLUCLP   

              destructionPercentage       duration        order       stars  \
attack_num                        1     2        1      2     1     2     1   
member_name                                                                   
Drippy                        100.0   NaN     93.0    NaN  10.0   NaN   3.0   
MeekMulli                      91.0   NaN    161.0    NaN  18.0   NaN   1.0   
Shamrock 2.0                   97.0   NaN    176.0    NaN  16.0   NaN   2.0   
Wreckit Ralph                 100.0   NaN    142.0    NaN   9.0   NaN   3.0   
winston                        94.0  75.0    129.0  157.0  11.0  17.0   2.0   

                    
attack_num       2  
member_name         
Drippy         NaN  
MeekMulli      NaN  
Shamrock 2.0   NaN  
Wreckit Ralph  NaN  
winston        1.0

In [11]:
pivot_df.columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

In [12]:
def extract_text_and_number(column_name):
    match = re.match(r'(.+?)_(\d+)', column_name)
    return (int(match.group(2)), match.group(1)) if match else (float('inf'), column_name)


In [13]:
sorted_columns = sorted(pivot_df.columns, key=lambda x: extract_text_and_number(x))
sorted_columns

['attackerTag_1',
 'defenderTag_1',
 'destructionPercentage_1',
 'duration_1',
 'order_1',
 'stars_1',
 'attackerTag_2',
 'defenderTag_2',
 'destructionPercentage_2',
 'duration_2',
 'order_2',
 'stars_2']

In [14]:
pivot_df = pivot_df[sorted_columns]

In [15]:
final_df = pivot_df.join(total_attacks_df)
final_df

,attackerTag_1,defenderTag_1,destructionPercentage_1,duration_1,order_1,stars_1,attackerTag_2,defenderTag_2,destructionPercentage_2,duration_2,order_2,stars_2,total_attacks
member_name,,,,,,,,,,,,,
Drippy,#LPG2LYYGJ,#G28LQQGL8,100.0,93.0,10.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1
MeekMulli,#8CRG2JR9,#QR9GLUCLP,91.0,161.0,18.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
Shamrock 2.0,#9GGVU8C8,#2QQ9RQJ0P,97.0,176.0,16.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1
Wreckit Ralph,#GPRURPLRG,#99PY8V9C2,100.0,142.0,9.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1
winston,#GYQ09LYRR,#URUQ92R,94.0,129.0,11.0,2.0,#GYQ09LYRR,#QR9GLUCLP,75.0,157.0,17.0,1.0,2


In [16]:
new_name_df = final_df
new_name_df['Avg %'] = new_name_df[['destructionPercentage_1', 'destructionPercentage_2']].mean(axis=1)
new_name_df['Total Stars'] = new_name_df[["stars_1", "stars_2"]].sum(axis=1)

In [17]:
new_order = ['total_attacks', 'Avg %', 'destructionPercentage_1', 'destructionPercentage_2', 
             'stars_1', 'stars_2', 'Total Stars']
new_name_df = new_name_df[new_order]
final_name_df = new_name_df.rename(columns={'destructionPercentage_1' : 'Attack 1 %', 'destructionPercentage_2' : 'Attack 2 %', 
                                            'stars_1' : 'Attack 1 Stars', 'stars_2' : 'Attack 2 Stars', 'total_attacks' : 'Total Attacks'})
final_name_df

,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars
member_name,,,,,,,
Drippy,1,100.0,100.0,NaN,3.0,NaN,3.0
MeekMulli,1,91.0,91.0,NaN,1.0,NaN,1.0
Shamrock 2.0,1,97.0,97.0,NaN,2.0,NaN,2.0
Wreckit Ralph,1,100.0,100.0,NaN,3.0,NaN,3.0
winston,2,84.5,94.0,75.0,2.0,1.0,3.0


In [18]:
try:
    total_members_df = total_members_df.set_index('name')
except KeyError:
    pass


In [19]:
final_name_df["Total Clan Stars"] = final_name_df['Total Stars'].sum()
final_name_df

,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars,Total Clan Stars
member_name,,,,,,,,
Drippy,1,100.0,100.0,NaN,3.0,NaN,3.0,12.0
MeekMulli,1,91.0,91.0,NaN,1.0,NaN,1.0,12.0
Shamrock 2.0,1,97.0,97.0,NaN,2.0,NaN,2.0,12.0
Wreckit Ralph,1,100.0,100.0,NaN,3.0,NaN,3.0,12.0
winston,2,84.5,94.0,75.0,2.0,1.0,3.0,12.0


In [20]:
combined_df = final_name_df.reindex(total_members_df.index)
combined_df.fillna(0, inplace=True)
combined_df.sort_values(["Total Attacks", 'Total Stars', 'Avg %'], ascending=False, inplace=True)

In [21]:
def colors(row):
    if row['Total Attacks'] == 2.0:
        color = "rgb(0,255,0)"
    elif row['Total Attacks'] == 1.0:
        color = "yellow"
    else:
        color = "red"
    return [f'color: {color}'] * len(row) 

In [22]:
styled_df = combined_df.style.apply(colors, axis=1).format('{:.1f}')
styled_df


,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars,Total Clan Stars
name,,,,,,,,
winston,2.0,84.5,94.0,75.0,2.0,1.0,3.0,12.0
Drippy,1.0,100.0,100.0,0.0,3.0,0.0,3.0,12.0
Wreckit Ralph,1.0,100.0,100.0,0.0,3.0,0.0,3.0,12.0
Shamrock 2.0,1.0,97.0,97.0,0.0,2.0,0.0,2.0,12.0
MeekMulli,1.0,91.0,91.0,0.0,1.0,0.0,1.0,12.0
mcnasty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NMbaseball22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ben Dover,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Luke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
